In [3]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


# Download data for BTC and QQQ (Nasdaq proxy)
btc_data = yf.download("BTC-USD", start="2025-06-17", interval="1m")
qqq_data = yf.download("QQQ", start="2025-06-17", interval="1m")

# Check what columns are available
print("BTC columns:", btc_data.columns)
print("QQQ columns:", qqq_data.columns)


/var/folders/ll/75nwzvxx25l4m2fksw9rb_2c0000gn/T/ipykernel_44779/3390812351.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  btc_data = yf.download("BTC-USD", start="2025-06-17", interval="1m")
[*********************100%***********************]  1 of 1 completed
/var/folders/ll/75nwzvxx25l4m2fksw9rb_2c0000gn/T/ipykernel_44779/3390812351.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  qqq_data = yf.download("QQQ", start="2025-06-17", interval="1m")
[*********************100%***********************]  1 of 1 completed

BTC columns: MultiIndex([( 'Close', 'BTC-USD'),
            (  'High', 'BTC-USD'),
            (   'Low', 'BTC-USD'),
            (  'Open', 'BTC-USD'),
            ('Volume', 'BTC-USD')],
           names=['Price', 'Ticker'])
QQQ columns: MultiIndex([( 'Close', 'QQQ'),
            (  'High', 'QQQ'),
            (   'Low', 'QQQ'),
            (  'Open', 'QQQ'),
            ('Volume', 'QQQ')],
           names=['Price', 'Ticker'])


In [4]:


btc = btc_data['Close']['BTC-USD']
qqq = qqq_data['Close']['QQQ']

steps = 120

# Combine into one DataFrame
data = pd.DataFrame({'BTC': btc, 'QQQ': qqq}).dropna()

# Compute daily percent change (returns)
returns = data.pct_change().dropna()

# Rolling correlation
rolling_corr = returns['BTC'].rolling(window=steps).corr(returns['QQQ'])

fig = go.Figure()

# Add the rolling correlation line
fig.add_trace(go.Scatter(
    x=rolling_corr.index,
    y=rolling_corr.values,
    mode='lines',
    name=f'{steps}-Period Rolling Correlation',
    line=dict(color='blue', width=2)
))

# Add horizontal line at zero
fig.add_hline(y=0, line_dash="dash", line_color="gray", 
              annotation_text="Zero Correlation")

# Update layout
fig.update_layout(
    title=f"{steps} Rolling Correlation: BTC vs Nasdaq (QQQ)",
    xaxis_title="Date",
    yaxis_title="Correlation",
    showlegend=True
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()

# Static correlation
corr = returns.corr().loc['BTC', 'QQQ']
print(f"Static correlation (daily returns): {corr:.2f}")

Static correlation (daily returns): 0.01
